In [4]:
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 5.0 MB/s eta 0:00:0000:01


In [5]:
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 7.1 MB/s eta 0:00:00


In [3]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.0 MB/s eta 0:00:0000:0100:01


In [1]:
!pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 27.6 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 23.3.2
    Uninstalling pip-23.3.2:
      Successfully uninstalled pip-23.3.2


In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import bz2
import csv
import re
import gc

from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer
from trl import setup_chat_format



import matplotlib.pyplot as plt
from tensorflow.python.keras import models, layers, optimizers
import tensorflow as tf
from tensorflow.keras.preprocessing import text
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score

from tensorflow.keras.preprocessing.text import Tokenizer

from transformers import PreTrainedTokenizerFast




%matplotlib inline


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory



# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

2024-05-04 07:12:48.774355: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-04 07:12:48.774446: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-04 07:12:48.908261: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [7]:
base_dir = ("/kaggle/input/amazonreviews/")

for fname in ['test.ft.txt.bz2', 'train.ft.txt.bz2']:
    ifname = base_dir + fname
    print(ifname)
    
    data = None
    with open(ifname, 'rb') as ifile:
        data = ifile.read()
    print('Data read, {} bytes'.format(len(data)))

/kaggle/input/amazonreviews/test.ft.txt.bz2
Data read, 52653659 bytes
/kaggle/input/amazonreviews/train.ft.txt.bz2
Data read, 464275989 bytes


In [8]:
def get_labels_and_texts(file):
    labels = []
    texts = []
    for line in bz2.BZ2File(file):
        x = line.decode("utf-8")
        labels.append(int(x[9]) - 1)
        texts.append(x[10:].strip())
    return np.array(labels), texts
train_labels, train_texts = get_labels_and_texts(base_dir + 'train.ft.txt.bz2')
test_labels, test_texts = get_labels_and_texts(base_dir + 'test.ft.txt.bz2')

In [9]:
print("test_labels:", test_labels)
print("train_labels:", train_labels)
print(type(train_labels)) # numpy
print(type(test_labels)) # numpy


print("train_texts 길이:", len(train_texts)) # 3600000
print("test_texts 길이:", len(test_texts)) # 400000

test_labels: [1 1 0 ... 0 1 0]
train_labels: [1 1 1 ... 0 0 1]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
train_texts 길이: 3600000
test_texts 길이: 400000


In [10]:
type(train_texts) # list
type(train_labels) # numpy

# type(train_labels) # numpy
train_labels2 = train_labels.tolist()
type(train_labels2) # list
train_labels3 = list(map(str, train_labels2))
type(train_labels3) # list

list

In [11]:
type(test_texts) # list
type(test_labels) # numpy

# type(train_labels) # numpy
test_labels2 = test_labels.tolist()
type(test_labels2) # list
test_labels3 = list(map(str, test_labels2))
type(test_labels3) # list

list

In [12]:
train_df = pd.Series([train_texts, train_labels3], index=['texts', 'labels'])
train_df

texts     [Stuning even for the non-gamer: This sound tr...
labels    [1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, ...
dtype: object

In [13]:
test_df = pd.Series([test_texts, test_labels3], index=['texts', 'labels'])
test_df

texts     [Great CD: My lovely Pat has one of the GREAT ...
labels    [1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, ...
dtype: object

In [14]:
combined_data = []
for text, label in zip(train_df.texts, train_df.labels):
    combined_data.append((text.strip("'"), label.replace(',', '')))
    

# Convert to DataFrame
train = pd.DataFrame(combined_data, columns=['texts', 'labels'])

In [15]:
# Print the DataFrame
# train.head()
# train.tail()  # 3599999
len(train)  # 3600000

3600000

In [16]:
combined_data2 = []
for text, label in zip(test_df.texts, test_df.labels):
    combined_data2.append((text.strip("'"), label.replace(',', '')))
    

# Convert to DataFrame
test = pd.DataFrame(combined_data2, columns=['texts', 'labels'])

# Print the DataFrame
test.tail()

,texts,labels
399995,Unbelievable- In a Bad Way: We bought this Tho...,0
399996,"Almost Great, Until it Broke...: My son reciev...",0
399997,Disappointed !!!: I bought this toy for my son...,0
399998,Classic Jessica Mitford: This is a compilation...,1
399999,"Comedy Scene, and Not Heard: This DVD will be ...",0


In [17]:
# Print the DataFrame
# test.head()
# test.tail()  # 399999
len(test) # 400000

400000

In [18]:
train_data = train.drop_duplicates(inplace=False)
train_data = train_data.dropna(how='any')
train_data

,texts,labels
0,Stuning even for the non-gamer: This sound tra...,1
1,The best soundtrack ever to anything.: I'm rea...,1
2,Amazing!: This soundtrack is my favorite music...,1
3,Excellent Soundtrack: I truly like this soundt...,1
4,"Remember, Pull Your Jaw Off The Floor After He...",1
...,...,...
3599995,Don't do it!!: The high chair looks great when...,0
3599996,"Looks nice, low functionality: I have used thi...",0
3599997,"compact, but hard to clean: We have a small ho...",0
3599998,what is it saying?: not sure what this book is...,0


In [19]:
# train_data = train_data.astype({'texts':'str','labels':'int32'})
# print(train_data.dtypes)

In [20]:
train_data.groupby(by=['labels']).count()

,texts
labels,
0,1800000
1,1800000


In [21]:
test_data = test.drop_duplicates(inplace=False)
test_data = test_data.dropna(how='any')
test_data

,texts,labels
0,Great CD: My lovely Pat has one of the GREAT v...,1
1,One of the best game music soundtracks - for a...,1
2,Batteries died within a year ...: I bought thi...,0
3,"works fine, but Maha Energy is better: Check o...",1
4,Great for the non-audiophile: Reviewed quite a...,1
...,...,...
399995,Unbelievable- In a Bad Way: We bought this Tho...,0
399996,"Almost Great, Until it Broke...: My son reciev...",0
399997,Disappointed !!!: I bought this toy for my son...,0
399998,Classic Jessica Mitford: This is a compilation...,1


In [22]:
# test_data = test_data.astype({'texts':'str','labels':'int32'})
# print(test_data.dtypes)

In [23]:
test_data.groupby(by=['labels']).count()

,texts
labels,
0,200000
1,200000


In [24]:
X_train = train_data.texts
X_test = train_data.labels
y_test = test_data.texts
y_true = test_data.labels

In [25]:
X_test.iloc[0]

'1'

### Load the model

In [26]:
# !pip install -q -U "transformers==4.36.2" "datasets==2.16.1" "accelerate==0.26.1" "bitsandbytes==0.42.0"

In [27]:
# !pip install --upgrade transformers

In [28]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"working on {device}")

working on cuda:0


In [30]:
import time
import torch
import transformers
from transformers import (AutoModelForCausalLM, 
                          AutoTokenizer, 
                          BitsAndBytesConfig, 
                          TrainingArguments, 
                          pipeline, 
                          logging)
from sklearn.metrics import (accuracy_score, 
                             classification_report, 
                             confusion_matrix)
from sklearn.model_selection import train_test_split


from IPython.display import display, Markdown

import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

In [31]:
model_name = "/kaggle/input/llama-2/pytorch/7b-hf/1"

compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, 
    bnb_4bit_quant_type="nf4", 
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=True,
)

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [32]:
tokenizer = AutoTokenizer.from_pretrained(model_name, 
                                          trust_remote_code=True,
                                         )
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [33]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=device,
    torch_dtype=compute_dtype,
    quantization_config=bnb_config, 
    # early_stopping = early_stopping
)

model.config.use_cache = False
model.config.pretraining_tp = 1


model, tokenizer = setup_chat_format(model, tokenizer)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuratio

In [34]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32003, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): Lla

In [35]:
def predict(test, model, tokenizer):
    y_pred = []
    for i in tqdm(range(len(X_test))):
        prompt = X_test.iloc[i]     # ["labels"]  # texts
        pipe = pipeline(task="text-generation", 
                        model=model, 
                        tokenizer=tokenizer, 
                        max_new_tokens = 1, 
                        temperature = 0.0,
                       )
        result = pipe(prompt)
        answer = result[0]['generated_text'].split("=")[-1]
        if "positive" in answer:
            y_pred.append("positive")
        elif "negative" in answer:
            y_pred.append("negative")
        elif "neutral" in answer:
            y_pred.append("neutral")
        else:
            y_pred.append("none")
    return y_pred

In [36]:
def evaluate(y_true, y_pred):
    labels = ['1', '0']
    mapping = {'positive': 1, 'negative': 0}
    def map_func(x):
        return mapping.get(x, 1)
    
    y_true = np.vectorize(map_func)(y_true)
    y_pred = np.vectorize(map_func)(y_pred)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
    print(f'Accuracy: {accuracy:.3f}')
    
    # Generate accuracy report
    unique_labels = set(y_true)  # Get unique labels
    
    for label in unique_labels:
        label_indices = [i for i in range(len(y_true)) 
                         if y_true[i] == label]
        label_y_true = [y_true[i] for i in label_indices]
        label_y_pred = [y_pred[i] for i in label_indices]
        accuracy = accuracy_score(label_y_true, label_y_pred)
        print(f'Accuracy for label {label}: {accuracy:.3f}')
        
    # Generate classification report
    class_report = classification_report(y_true=y_true, y_pred=y_pred)
    print('\nClassification Report:')
    print(class_report)
    
    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=[0, 1, 2])
    print('\nConfusion Matrix:')
    print(conf_matrix)

In [ ]:
y_pred = predict(test_data.texts, model, tokenizer)

  0%|          | 0/3600000 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
  1%|          | 27044/3600000 [1:43:22<227:30:58,  4.36it/s]

In [ ]:
evaluate(y_true, y_pred)

In [ ]:
output_dir="trained_weigths"

peft_config = LoraConfig(
        lora_alpha=16, 
        lora_dropout=0.1,
        r=64,
        bias="none",
        target_modules="all-linear",
        task_type="CAUSAL_LM",
)

training_arguments = TrainingArguments(
    output_dir=output_dir,                    # directory to save and repository id
    num_train_epochs=3,                       # number of training epochs
    per_device_train_batch_size=1,            # batch size per device during training
    gradient_accumulation_steps=8,            # number of steps before performing a backward/update pass
    gradient_checkpointing=True,              # use gradient checkpointing to save memory
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=25,                         # log every 10 steps
    learning_rate=2e-4,                       # learning rate, based on QLoRA paper
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,                        # max gradient norm based on QLoRA paper
    max_steps=-1,
    warmup_ratio=0.03,                        # warmup ratio based on QLoRA paper
    group_by_length=True,
    lr_scheduler_type="cosine",               # use cosine learning rate scheduler
    report_to="tensorboard",                  # report metrics to tensorboard
    evaluation_strategy="epoch"               # save checkpoint every epoch
)

trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=train_data,
    eval_dataset=eval_data,
    peft_config=peft_config,
    dataset_text_field="text",
    tokenizer=tokenizer,
    max_seq_length=1024,
    packing=False,
    dataset_kwargs={
        "add_special_tokens": False,
        "append_concat_token": False,
    }
)

In [ ]:
# Train model
trainer.train()

In [ ]:
# Save trained model and tokenizer
trainer.save_model()
tokenizer.save_pretrained(output_dir)